In [9]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'blake-griffin', 'hasheem-thabeet', 'james-harden', 'tyreke-evans', 'jonny-flynn', 'stephen-curry',  
    'jordan-hill', 'demar-derozan', 'terrence-williams', 'gerald-henderson-2', 'tyler-hansbrough', 
    'earl-clark', 'austin-daye', 'james-johnson-2', 'jrue-holiday', 'ty-lawson', 'jeff-teague', 
    'eric-maynor', 'darren-collison', 'demarre-carroll', 'wayne-ellington', 'toney-douglas', 'jeff-ayres', 
    'jermaine-taylor', 'dante-cunningham', 'dajuan-summers', 'sam-young',  'dejuan-blair', 'jon-brockman', 
    'derrick-brown', 'jodie-meeks', 'patrick-beverly', 'chase-budinger', 'nick-calathes', 'byron-mullins',
    'danny-green', 'taylor-griffin', 'goran-suton', 'jack-mcclinton', 'aj-price', 'patrick-mills', 
    'taj-gibson', 'robert-dozier', 'marcus-thornton']
   
player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2009')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2009'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/30))*2.95), 2)
        year_list.append(year)
        
    except:
        age_list.append(22)
        year_list.append(.85)

year_list[-1] = .79
age_list[-1] = 22
table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)

table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

table["Player Grade"] = ((table['PTS']) + (table['TRB']*1.5) + (table['AST']*2) +
(table['BLK']*3) + (table['STL']*3) + (table['3P']*5)+ (table['FT%']*7) + (table['SOS']) + 
(table['3P%']*10)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.50
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['TOV']
del table['PF']

table

,Name,Season,Age,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,PTS,,Player Grade
1,Stephen Curry,2008-09,21.3,Davidson,-3.33,34,33.7,.454,3.8,9.9,0.387,0.876,4.4,5.6,2.5,0.2,28.6,,103.4
2,Blake Griffin,2008-09,20.3,Oklahoma,6.88,35,33.3,.654,0.1,0.2,0.375,0.590,14.4,2.3,1.1,1.2,22.7,,101.3
3,James Harden,2008-09,19.9,Arizona State,7.82,35,35.8,.489,1.7,4.7,0.356,0.756,5.6,4.2,1.7,0.3,20.1,,101.1
4,Nick Calathes,2008-09,20.4,Florida,3.88,36,33.3,.482,2.0,5.2,0.390,0.707,5.3,6.4,1.9,0.2,17.2,,94.4
5,Jonny Flynn,2008-09,20.4,Syracuse,9.48,38,37.3,.460,1.2,3.7,0.317,0.786,2.7,6.7,1.4,0.2,17.4,,90.0
6,Chase Budinger,2008-09,21.1,Arizona,8.80,35,37.6,.480,1.9,4.8,0.399,0.801,6.2,3.4,1.4,0.5,18.0,,89.4
7,Tyreke Evans,2008-09,19.8,Memphis,4.63,37,29.0,.455,0.9,3.4,0.274,0.711,5.4,3.9,2.1,0.8,17.1,,87.8
8,Jack Mcclinton,2008-09,22.0,Miami (FL),9.04,32,32.2,.449,3.2,7.0,0.453,0.885,3.1,2.8,0.8,0.0,19.3,,86.3
9,Jeff Teague,2008-09,21.1,Wake Forest,7.36,31,32.0,.485,1.5,3.3,0.441,0.817,3.3,3.5,1.9,0.6,18.8,,83.5
10,Ty Lawson,2008-09,21.7,UNC,8.35,35,29.9,.532,1.5,3.1,0.472,0.798,3.0,6.6,2.1,0.1,16.6,,82.5
